In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore')
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score,RandomizedSearchCV
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,accuracy_score


In [2]:
np.random.seed(1)

In [3]:
def sigmoid(x):
     return 1 / (1 + np.exp(-x))

In [4]:
def ReLU(x):
    return x  *  (x  >  0)

In [5]:
def sigmoid_deriv(x):
    return sigmoid(x)*(1 - sigmoid(x))

In [6]:
def ReLU_deriv(x):
        return (x>0).astype(np.int0)

In [7]:
# преобразование массива в бинарный вид результатов
def to_one_hot(Y):
    n_col = np.amax(Y) 
    n = len(Y)
    binarized = np.zeros((n, n_col))
    for i in range(n):
        binarized[i, Y[i]-1] = 1.
    return binarized

In [8]:
# преобразование массива в необходимый вид
def from_one_hot(layer):
    arr = np.argmax(layer,axis=1)+1
    return arr.reshape(-1,1)

In [9]:
def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

In [10]:
def replace_target(df,name):
    df_1 = df.copy()
    d = { j:(i+1) for i,j in enumerate(iris_data['Species'].unique())}
    df_1[name] = df_1[name].map(d)
    return df_1
    
    

In [11]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy,print_is = True):
   
    
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        estimator.fit(x_train, y_train) 

        y_valid_pred = estimator.predict_proba(x_valid)[:,1]
        y_train_pred = estimator.predict_proba(x_train)[:,1]
        
        
        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        if print_is:
            print(msg)
            print("="*69)
        estimators.append(estimator)
    oof_score = metric(y, oof_predictions)
    if print_is:
        print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
        print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
        print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions  

In [12]:
def make_cross_validation_classif_multi_n(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy,print_is = True):
   
    
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros((X.shape[0],y.shape[1]))   

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        estimator.fit(x_train, y_train) 

        y_valid_pred = estimator.predict_proba(x_valid)
        y_train_pred = estimator.predict_proba(x_train)
        
        
        fold_train_scores.append(metric(y_train, y_train_pred,multi_class = 'ovr'))
        fold_valid_scores.append(metric(y_valid, y_valid_pred,multi_class = 'ovr'))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        if print_is:
            print(msg)
            print("="*69)
        estimators.append(estimator)
    oof_score = metric(y, oof_predictions,multi_class = 'ovr')
    if print_is:
        print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
        print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
        print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions  

In [13]:
def make_cross_validation_classif_multi(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy,print_is = True):
   
    
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros((X.shape[0],y.nunique())) 

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        estimator.fit(x_train, y_train) 

        y_valid_pred = estimator.predict_proba(x_valid)
        y_train_pred = estimator.predict_proba(x_train)
        
        
        fold_train_scores.append(metric(y_train, y_train_pred,multi_class = 'ovr'))
        fold_valid_scores.append(metric(y_valid, y_valid_pred,multi_class = 'ovr'))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        if print_is:
            print(msg)
            print("="*69)
        estimators.append(estimator)
    oof_score = metric(y, oof_predictions,multi_class = 'ovr')
    if print_is:
        print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
        print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
        print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions  

In [14]:
def serch_best_params(param_grid,estimator,metric,cv_strategy,x_train,y_train,res_func=make_cross_validation,k=10):    
    param_name =list(param_grid.keys())
    n = len(param_grid[param_name[0]])
    list_params=[]
    list_res=[]

    for j in range(n):
        qu=0
        for i in range(k*n):
            work_param ={p:np.random.choice(param_grid[p]) for p in param_name}
            if work_param in list_params:
                continue 
            list_res.append(res_func(x_train,y_train,\
                        estimator.set_params(**work_param),metric,cv_strategy,print_is = False)[1])    
            list_params.append(work_param)
            qu+=1
        if qu==0:
            continue
        min_param = list_params[list_res.index(min(list_res[-qu:]))]
        for min_key in min_param:
            param_grid[min_key].remove(min_param[min_key])
            
    max_res= max(list_res)
    best_params = list_params[list_res.index(max_res)]
    return max_res,best_params

In [15]:
class Neural_network_hide_1:
    def __init__(self,eta,num_neurons,num_iter,activ_func,start):
        self.base = start
        self.eta = eta
        self.num_neurons = num_neurons
        self.num_iter = num_iter
        self.activ_func,self.deriv_activ_func = activ_func
     
    def fit(self,X,y):
        self.w0 = np.zeros((X.shape[1], self.num_neurons))
        self.w0 = self.w0 + self.base
        self.w1 = np.zeros((self.num_neurons, y.shape[1]))
        self.w1 = self.w1 +self.base
        
        for i in range(self.num_iter): 
            layer0,layer1,layer2 = self.direct(X)
            self.opposite(y,layer0,layer1,layer2)

    def direct(self,X):
        layer0 = X
        layer1 = self.activ_func(np.dot(layer0, self.w0))
        layer2 = self.activ_func(np.dot(layer1, self.w1))
        return layer0,layer1,layer2
    
    def opposite(self,y,layer0,layer1,layer2):
        layer2_error = y - layer2
        layer2_delta = layer2_error * self.deriv_activ_func(layer2)

        layer1_error = layer2_delta.dot(self.w1.T)
        layer1_delta = layer1_error * self.deriv_activ_func(layer1)

        self.w1 += layer1.T.dot(layer2_delta) * self.eta
        self.w0 += layer0.T.dot(layer1_delta) * self.eta
    
    def  predict_proba(self,X):
        layer0,layer1,layer2 = self.direct(X)
        sum_labals = (np.sum(layer2,axis=1))
        sum_labals[sum_labals==0] = 1 
        sum_labals = sum_labals.reshape(-1,1)              
        return layer2/sum_labals
        
        
    
    
    
        
    

In [16]:
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=100)

In [17]:
param_grid_lgbm   = {'n_estimators':[50,100,200,300,400,500,700,800,600,1000],'reg_alpha': [0.5,1,1.5,2,3,4,5,8,10,20],
               'reg_lambda': [0.5,1,20,50,100,150,300,500,550,600],
                'max_depth': [2,3,4,5,6,7,8,9,10,12],'min_child_samples':[5,10,15,20,25,30,35,40,45,50],
               'num_leaves':[5,10,25,30,35,40,45,50,55,60]}

param_grid_cb   = {'n_estimators':[50,100,200,300,400,500,700,800,600,1000],'l2_leaf_reg': [0.5,1,2,5,10,15,20,30,40,50],
                'max_depth': [2,3,4,5,6,7,8,9,10,12],'min_child_samples':[5,10,15,20,25,30,35,40,45,50],
                'max_bin':[5,10,25,30,35,40,45,50,55,60]}
param_grid_rf = {'n_estimators': [50,100,200,300,400,500,700,800,600,1000], 'min_samples_split': [2,4,6,8,10,12,14,16,18,25]
                 , 'min_samples_leaf': [2,4,6,8,10,12,14,16,18,25]}


param_grid_lr ={'C': [0.1,1,1.5,2,2.5,3,4,5,6,7] ,'penalty': ['l1','l2']}

params=[param_grid_lr,
        param_grid_rf,
        param_grid_cb,
        param_grid_lgbm]    

### Посмотрим какие результаты дадут классические модели

In [18]:
estimators= [LogisticRegression(max_iter=1000,solver = 'liblinear'),
             RandomForestClassifier(n_jobs=-1),
             cb.CatBoostClassifier(thread_count=15,early_stopping_rounds=90,verbose=False),
             lgb.LGBMClassifier(n_jobs=-1)]

In [19]:
iris_data = pd.read_csv("Iris.csv")

In [20]:
iris_data

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [21]:
iris_data_work = replace_target(iris_data,'Species')

In [22]:
iris_data_work.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,1
1,2,4.9,3.0,1.4,0.2,1
2,3,4.7,3.2,1.3,0.2,1
3,4,4.6,3.1,1.5,0.2,1
4,5,5.0,3.6,1.4,0.2,1


In [23]:
feath =np.array(iris_data_work.columns[1:5])

In [24]:
feath

array(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'],
      dtype=object)

In [25]:
target= 'Species'

In [26]:
X= normalize(iris_data_work[feath].to_numpy())

In [27]:
X

array([[0.80377277, 0.55160877, 0.22064351, 0.0315205 ],
       [0.82813287, 0.50702013, 0.23660939, 0.03380134],
       [0.80533308, 0.54831188, 0.2227517 , 0.03426949],
       [0.80003025, 0.53915082, 0.26087943, 0.03478392],
       [0.790965  , 0.5694948 , 0.2214702 , 0.0316386 ],
       [0.78417499, 0.5663486 , 0.2468699 , 0.05808704],
       [0.78010936, 0.57660257, 0.23742459, 0.0508767 ],
       [0.80218492, 0.54548574, 0.24065548, 0.0320874 ],
       [0.80642366, 0.5315065 , 0.25658935, 0.03665562],
       [0.81803119, 0.51752994, 0.25041771, 0.01669451],
       [0.80373519, 0.55070744, 0.22325977, 0.02976797],
       [0.786991  , 0.55745196, 0.26233033, 0.03279129],
       [0.82307218, 0.51442011, 0.24006272, 0.01714734],
       [0.8025126 , 0.55989251, 0.20529392, 0.01866308],
       [0.81120865, 0.55945424, 0.16783627, 0.02797271],
       [0.77381111, 0.59732787, 0.2036345 , 0.05430253],
       [0.79428944, 0.57365349, 0.19121783, 0.05883625],
       [0.80327412, 0.55126656,

In [28]:
Y= to_one_hot(iris_data_work[target].to_numpy())

In [29]:
Y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [30]:
ml_result=pd.DataFrame(columns=['model','params','result'])

In [31]:
for i,estimator in enumerate(estimators):
    ml_result.loc[i,'model'] = str(estimator)
    model=RandomizedSearchCV(estimator=estimator,cv=cv_strategy,\
        param_distributions = params[i],n_jobs = -1,n_iter = 1000).fit(iris_data_work[feath],iris_data_work[target])
    best_params = model.best_params_
    max_res = make_cross_validation_classif_multi(iris_data_work[feath],\
                        iris_data_work[target],model.best_estimator_,roc_auc_score,cv_strategy=cv_strategy)[1]
    ml_result.loc[i,'result'] = max_res
    ml_result.loc[i,'params'] = str(best_params)
    print(ml_result)
    
    

Fold: 1, train-observations = 120, valid-observations = 30
train-score = 0.9947, valid-score = 1.0
Fold: 2, train-observations = 120, valid-observations = 30
train-score = 0.9921, valid-score = 1.0
Fold: 3, train-observations = 120, valid-observations = 30
train-score = 1.0, valid-score = 0.972
Fold: 4, train-observations = 120, valid-observations = 30
train-score = 0.9944, valid-score = 1.0
Fold: 5, train-observations = 120, valid-observations = 30
train-score = 0.9937, valid-score = 1.0
CV-results train: 0.995 +/- 0.003
CV-results valid: 0.9944 +/- 0.011
OOF-score = 0.9929
                                               model  \
0  LogisticRegression(max_iter=1000, solver='libl...   

                      params    result  
0  {'penalty': 'l1', 'C': 5}  0.992867  
Fold: 1, train-observations = 120, valid-observations = 30
train-score = 0.9994, valid-score = 0.9885
Fold: 2, train-observations = 120, valid-observations = 30
train-score = 0.9993, valid-score = 1.0
Fold: 3, train-observa

In [46]:
ml_result.sort_values('result',ascending=False,inplace=True)
ml_result

,model,params,result
2,<catboost.core.CatBoostClassifier object at 0x...,"{'n_estimators': 50, 'min_child_samples': 30, ...",0.996333
1,RandomForestClassifier(n_jobs=-1),"{'n_estimators': 200, 'min_samples_split': 2, ...",0.994133
0,"LogisticRegression(max_iter=1000, solver='libl...","{'penalty': 'l1', 'C': 5}",0.992867
3,LGBMClassifier(),"{'reg_lambda': 1, 'reg_alpha': 2, 'num_leaves'...",0.9866


### Решение задачи с помощью нейронной сети

In [34]:
params_grig1={'eta' : [0.001,0.01,0.1,1],
             'num_neurons' :[3,4, 5,8,10,12,15,20,25,30],
             'num_iter' : [1000,10000,30000],
             'activ_func' : (sigmoid,sigmoid_deriv),
             'start' : 0.3}
params_grig2={'eta' : '',
             'num_neurons' :'',
             'num_iter' : '',
             'activ_func' : (sigmoid,sigmoid_deriv),
             'start' : [0,0.1,-0.1,-0.3,0.5,0.7,1,2,5,-2]}
params_grig3={'eta' : [0.001,0.01,0.1,1],
             'num_neurons' :[2,3, 5,8,10,12,15,20,25,30],
             'num_iter' : [5000,50000,100000],
             'activ_func' : (ReLU,ReLU_deriv),
             'start' : 0.3}
params_grig4={'eta' : '',
             'num_neurons' :'',
             'num_iter' : '',
             'activ_func' : (ReLU,ReLU_deriv),
             'start' : [0,0.1,-0.1,-0.3,0.5,0.7,1,2,5,-2]}


In [35]:
res_sig = pd.DataFrame(columns=['eta','num_neurons','num_iter','activ_func','start','roc_auc'])
my_params={}
for eta  in params_grig1['eta']:
    for num_neurons in  params_grig1['num_neurons']:
        for num_iter in  params_grig1['num_iter']:
            my_params = {'eta' : eta,
                         'num_neurons' :num_neurons,
                         'num_iter' : num_iter,
                         'activ_func' : params_grig1['activ_func'],
                         'start' : params_grig1['start']}
            my_net = Neural_network_hide_1(**my_params) 
            my_params['roc_auc']  = make_cross_validation_classif_multi_n(X, Y,my_net,roc_auc_score,cv_strategy,print_is =False)[1]
            my_params['activ_func'] = str(params_grig1['activ_func'])[10:18]            
            res_sig = res_sig.append(my_params,ignore_index=True)               
                    
        

In [36]:
res_sig.sort_values('roc_auc',ascending=False,inplace=True)
res_sig.head(20)

,eta,num_neurons,num_iter,activ_func,start,roc_auc
80,0.10,15,30000,sigmoid,0.3,0.997733
49,0.01,15,10000,sigmoid,0.3,0.997533
88,0.10,30,10000,sigmoid,0.3,0.997200
59,0.01,30,30000,sigmoid,0.3,0.995933
53,0.01,20,30000,sigmoid,0.3,0.995867
76,0.10,12,10000,sigmoid,0.3,0.995800
58,0.01,30,10000,sigmoid,0.3,0.995600
79,0.10,15,10000,sigmoid,0.3,0.995333
52,0.01,20,10000,sigmoid,0.3,0.995067
73,0.10,10,10000,sigmoid,0.3,0.994133


In [37]:
params_grig2['eta']=res_sig.iloc[0].eta
params_grig2['num_neurons']=res_sig.iloc[0].num_neurons
params_grig2['num_iter']=res_sig.iloc[0].num_iter
my_params={}
for start  in params_grig2['start']:
    my_params = params_grig2.copy()
    my_params['start'] = start
    my_net = Neural_network_hide_1(**my_params) 
    my_params['roc_auc']  = make_cross_validation_classif_multi_n(X, Y,my_net,roc_auc_score,cv_strategy,print_is =False)[1]
    my_params['activ_func'] = str(params_grig1['activ_func'])[10:18]            
    res_sig = res_sig.append(my_params,ignore_index=True)      

In [38]:
res_sig.sort_values('roc_auc',ascending=False,inplace=True)
res_sig.head(20)

,eta,num_neurons,num_iter,activ_func,start,roc_auc
0,0.10,15,30000,sigmoid,0.3,0.997733
1,0.01,15,10000,sigmoid,0.3,0.997533
2,0.10,30,10000,sigmoid,0.3,0.997200
124,0.10,15,30000,sigmoid,0.5,0.996067
3,0.01,30,30000,sigmoid,0.3,0.995933
4,0.01,20,30000,sigmoid,0.3,0.995867
5,0.10,12,10000,sigmoid,0.3,0.995800
6,0.01,30,10000,sigmoid,0.3,0.995600
7,0.10,15,10000,sigmoid,0.3,0.995333
8,0.01,20,10000,sigmoid,0.3,0.995067


In [39]:
res_relu = pd.DataFrame(columns=['eta','num_neurons','num_iter','activ_func','start','roc_auc'])
my_params={}
for eta  in params_grig3['eta']:
    for num_neurons in  params_grig3['num_neurons']:
        for num_iter in  params_grig3['num_iter']:
            my_params = {'eta' : eta,
                         'num_neurons' :num_neurons,
                         'num_iter' : num_iter,
                         'activ_func' : params_grig3['activ_func'],
                         'start' : params_grig3['start']}
            my_net = Neural_network_hide_1(**my_params) 
            my_params['roc_auc']  = make_cross_validation_classif_multi_n(X, Y,my_net,roc_auc_score,cv_strategy,print_is =False)[1]
            my_params['activ_func'] = str(params_grig3['activ_func'])[10:18]            
            res_relu = res_relu.append(my_params,ignore_index=True)       



In [40]:
res_relu.sort_values('roc_auc',ascending=False,inplace=True)
res_relu.head(20)

,eta,num_neurons,num_iter,activ_func,start,roc_auc
27,0.001,30,5000,ReLU at,0.3,0.997467
24,0.001,25,5000,ReLU at,0.3,0.996267
21,0.001,20,5000,ReLU at,0.3,0.996133
0,0.001,2,5000,ReLU at,0.3,0.996000
12,0.001,10,5000,ReLU at,0.3,0.996000
18,0.001,15,5000,ReLU at,0.3,0.995867
6,0.001,5,5000,ReLU at,0.3,0.995867
9,0.001,8,5000,ReLU at,0.3,0.995867
5,0.001,3,100000,ReLU at,0.3,0.995867
15,0.001,12,5000,ReLU at,0.3,0.995733


In [41]:
params_grig4['eta']=res_relu.iloc[0].eta
params_grig4['num_neurons']=res_relu.iloc[0].num_neurons
params_grig4['num_iter']=res_relu.iloc[0].num_iter
my_params={}
for start  in params_grig4['start']:
    my_params = params_grig4.copy()
    my_params['start'] = start
    my_net = Neural_network_hide_1(**my_params) 
    my_params['roc_auc']  = make_cross_validation_classif_multi_n(X, Y,my_net,roc_auc_score,cv_strategy,print_is =False)[1]
    my_params['activ_func'] = str(params_grig4['activ_func'])[10:18]            
    res_relu = res_relu.append(my_params,ignore_index=True) 

In [42]:
res_relu.sort_values('roc_auc',ascending=False,inplace=True)
res_relu.head(20)

,eta,num_neurons,num_iter,activ_func,start,roc_auc
0,0.001,30,5000,ReLU at,0.3,0.997467
123,0.001,30,5000,ReLU at,0.3,0.997467
1,0.001,25,5000,ReLU at,0.3,0.996267
2,0.001,20,5000,ReLU at,0.3,0.996133
3,0.001,2,5000,ReLU at,0.3,0.996000
4,0.001,10,5000,ReLU at,0.3,0.996000
5,0.001,15,5000,ReLU at,0.3,0.995867
6,0.001,5,5000,ReLU at,0.3,0.995867
7,0.001,8,5000,ReLU at,0.3,0.995867
8,0.001,3,100000,ReLU at,0.3,0.995867


In [43]:
res_sig.sort_values('roc_auc',ascending=False,inplace=True)
res_sig.head(20)

,eta,num_neurons,num_iter,activ_func,start,roc_auc
0,0.10,15,30000,sigmoid,0.3,0.997733
1,0.01,15,10000,sigmoid,0.3,0.997533
2,0.10,30,10000,sigmoid,0.3,0.997200
124,0.10,15,30000,sigmoid,0.5,0.996067
3,0.01,30,30000,sigmoid,0.3,0.995933
4,0.01,20,30000,sigmoid,0.3,0.995867
5,0.10,12,10000,sigmoid,0.3,0.995800
6,0.01,30,10000,sigmoid,0.3,0.995600
7,0.10,15,10000,sigmoid,0.3,0.995333
8,0.01,20,10000,sigmoid,0.3,0.995067


In [47]:
ml_result

,model,params,result
2,<catboost.core.CatBoostClassifier object at 0x...,"{'n_estimators': 50, 'min_child_samples': 30, ...",0.996333
1,RandomForestClassifier(n_jobs=-1),"{'n_estimators': 200, 'min_samples_split': 2, ...",0.994133
0,"LogisticRegression(max_iter=1000, solver='libl...","{'penalty': 'l1', 'C': 5}",0.992867
3,LGBMClassifier(),"{'reg_lambda': 1, 'reg_alpha': 2, 'num_leaves'...",0.9866
